In [34]:
from neo4j import GraphDatabase

user_id = 1110008753489760261
tweet_id = 1109874238398320640
politic_usa_id = 14792049

uri = 'bolt://localhost:7687'
driver = GraphDatabase.driver(uri)

In [35]:
def retrieve_all_tweet_stories():
    message = """MATCH p=(end:Tweet)-[:REPLIED_TO|RETWEETED_FROM*]->(start:Tweet {type: 'TWEET'})
                 WHERE NOT (start)-[:REPLIED_TO|RETWEETED_FROM]->() AND NOT ()-[:REPLIED_TO|RETWEETED_FROM]->(end)
                 WITH *, relationships(p) AS r
                 RETURN end.created_at, start.created_at, r"""
    records = driver.session().run(message).data()
    return records

In [69]:
def retrieve_tweet_story(tweet_id):
    message = """MATCH p=(end:Tweet)-[:REPLIED_TO|RETWEETED_FROM*]->(start:Tweet {type: 'TWEET'})
                 WHERE NOT (start)-[:REPLIED_TO|RETWEETED_FROM]->() AND NOT ()-[:REPLIED_TO|RETWEETED_FROM]->(end)
                 AND start.id = {id}
                 WITH *, relationships(p) AS r
                 RETURN end.created_at, start.created_at, r"""
    records = driver.session().run(message, id=tweet_id).data()
    return records

In [37]:
def retrieve_users_who_interacted_at_a_story(tweet_id):
    message = """MATCH u=(user:User)-[:TWEETED|POSTED_REPLY|POSTED_RETWEET]->(end:Tweet)-[:REPLIED_TO|RETWEETED_FROM*]->(start:Tweet)<-[:TWEETED]-(storyStarter:User)
                 WHERE start.id = {id} AND (end.type = 'RETWEET' OR end.type = 'REPLY') AND start.type = 'TWEET'
                 RETURN user, storyStarter"""
    records = driver.session().run(message, id=tweet_id).data()
    return records

In [38]:
def retrieve_stories_where_user_is_involved(user_id):
    message = """MATCH u=(user:User)-[:TWEETED|POSTED_REPLY|POSTED_RETWEET]->(end:Tweet)-[:REPLIED_TO|RETWEETED_FROM*]->(start:Tweet)<-[:TWEETED]-(storyStarter:User)
                 WHERE user.id = {id} or storyStarter.id = {id} AND (end.type = 'RETWEET' OR end.type = 'REPLY') AND start.type = 'TWEET'
                 WITH *, relationships(u) as re
                 RETURN end, re, start, user, storyStarter"""
    records = driver.session().run(message, id=user_id).data()
    return records

In [60]:
def retrieve_tweet_chain_ordered_by_time(tweet_id):
    # not needed, tweet chains are created in order in the first place
    print()

In [58]:
def retrieve_stories_made_by_user(user_id):
    message = """MATCH u=(user:User)-[:TWEETED]->(tweet:Tweet)
                 WHERE user.id = {id}
                 MATCH p=(end:Tweet)-[:REPLIED_TO|RETWEETED_FROM*]->(tweet:Tweet)
                 WHERE end.type = 'RETWEET' OR end.type = 'REPLY' AND tweet.type = 'TWEET'
                 RETURN u"""
    records = driver.session().run(message, id=user_id).value()
    return records

In [78]:
def retrieve_time_spread_of_tweet_story(tweet_id):
    records = retrieve_tweet_story(tweet_id)[0]
    start_time = records['start.created_at']
    end_time = records['end.created_at']
    message = "Original tweet was posted on %s. \nLast available reaction was posted on %s." % (start_time, end_time)
    
    return message

In [79]:
# print(retrieve_tweet_story(1109874238398320640))
# print(retrieve_users_who_interacted_at_a_story(1109874238398320640))
# for value in retrieve_stories_made_by_user(politic_usa_id):
#     print(value)
#     print("HELLO DARKNESS WORLD")
#     print("\n")

print(retrieve_time_spread_of_tweet_story(tweet_id))

Original tweet was posted on Sun Mar 24 17:46:53 EET 2019. 
Last available reaction was posted on Mon Mar 25 06:47:22 EET 2019.
